#### Sentinel Hub - Fetch images for fields

https://docs.sentinel-hub.com/api/latest/user-guides/beginners-guide/#python


https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request.html

In [ ]:
# Import modules
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from PIL import Image
import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import json
import datetime as dt
from math import ceil
import tensorflow as tf
import time
from sentinelhub import (SHConfig, 
                        SentinelHubCatalog, 
                        BBox, 
                        CRS, 
                        DataCollection, 
                        Geometry, 
                        filter_times, 
                        SentinelHubRequest, 
                        MimeType, 
                        SentinelHubDownloadClient)

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import plot_image

In [ ]:
# =============================================================================
# Read in the Catch Crop fields
# =============================================================================

# Path to GeoJSON
path_to_file = "Data/ccInfoGDF.geojson"
path_to_file = "C:/Users/morte/OneDrive - Syddansk Universitet/Speciale2023/Data/ccInfoGDF.geojson" # GeoJSON med alle typer efterafgrøder
path_to_file = f'D:/data/CatchCrop/ccInfoGDF.geojson' # GeoJSON med alle typer efterafgrøder
# path_to_file = "C:/Users/morte/OneDrive - Syddansk Universitet/Speciale2023/Data/ccNoAltInfoGDF.geojson" # GeoJSON kun med Målrettede Efterafgrøder

# Read in GeoJSON
with open(path_to_file, encoding='utf-8') as f:
    ccData = json.load(f)

    

In [ ]:
# =============================================================================
# Sentinel Hub
# =============================================================================

# Creates configuration file for SentinelHub service
config = SHConfig()

# Saves Client ID and Secret to configuration
# Credentials should be submitted in the follwong objects:
config.sh_client_id = '' 
config.sh_client_secret = ''

# Creates a catalog of Sentinel Hub services
catalog = SentinelHubCatalog(config=config)

collection_id = "sentinel-2-l2a"

# evalscript
evalscriptRGB = """
//VERSION=3

function setup() {
    return {
        input: [{
            bands: ["B04", "B03", "B02"],
            units: "REFLECTANCE" // default units
        }],
        output: { 
            id: 'default',
            bands: 3,
            sampleType: "AUTO" // default value - scales the output values from [0,1] to [0,255].
        }
    };
}

function evaluatePixel(sample) {
  return [sample.B02, 
          sample.B03, 
          sample.B04];
}
"""

evalscriptAllBands_L2A = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B02","B03","B04","B08","B8A","B11","B12","CLP", "CLM"],
                units: "DN"
            }],
            output: {
                bands: 9,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B02,
                sample.B03,
                sample.B04,
                sample.B08,
                sample.B8A,
                sample.B11,
                sample.B12,
                sample.CLP,
                sample.CLM];
    }
"""
# Band selection
# https://gisgeography.com/sentinel-2-bands-combinations/

# Color Infrared (B8, B4, B3)
# Sentinel 2 Color Infrared
# The color infrared band combination is meant to emphasize healthy and unhealthy vegetation. By using the near-infrared (B8) band, it’s especially good at reflecting chlorophyll. 
# This is why in a color infrared image, denser vegetation is red. But urban areas are white.

# Short-Wave Infrared (B12, B8A, B4)
# Sentinel 2 Shortwave Infrared
# The short-wave infrared band combination uses SWIR (B12), NIR (B8A), and red (B4). This composite shows vegetation in various shades of green. 
# In general, darker shades of green indicate denser vegetation. But brown is indicative of bare soil and built-up areas.

# Agriculture (B11, B8, B2)
# Sentinel 2 Agriculture
# The agriculture band combination uses SWIR-1 (B11), near-infrared (B8), and blue (B2). It’s mostly used to monitor the health of crops because of how it uses short-wave and near-infrared. 
# Both these bands are particularly good at highlighting dense vegetation that appears as dark green.

# Vegetation Index (B8-B4)/(B8+B4)
# Sentinel 2 Vegetation Index
# Because near-infrared (which vegetation strongly reflects) and red light (which vegetation absorbs), the vegetation index is good for quantifying the amount of vegetation. 
# The formula for the normalized difference vegetation index is (B8-B4)/(B8+B4). While high values suggest dense canopy, low or negative values indicate urban and water features.

# Moisture Index (B8A-B11)/(B8A+B11)
# Sentinel 2 Moisture Index
# The moisture index is ideal for finding water stress in plants. It uses the short-wave and near-infrared to generate an index of moisture content. 
# In general, wetter vegetation has higher values. But lower moisture index values suggest plants are under stress from insufficient moisture.



In [ ]:
# =============================================================================
# Sentinel 2 - L2A
# =============================================================================
# Subset number of fields to process
k = 500

idxStart = 5000
idxEnd = 7943

ccFields = ccData['features'][idxStart:idxEnd] # To test with

# Create a dict of processed fields
processedFields = {'ID': []}

fieldData = []

# Create a list of periods in 2021 and 2022
dtPeriods2021 = pd.date_range(start="2021-08-01T00:00:00Z",end="2021-11-01T00:00:00Z", periods = 10).to_pydatetime().tolist()
dtPeriods2022 = pd.date_range(start="2022-08-01T00:00:00Z",end="2022-11-01T00:00:00Z", periods = 10).to_pydatetime().tolist()

# Create a list of intervals in 2021 and 2022
dtIntervals2021 = []
dtIntervals2022 = []

for y in range(len(dtPeriods2021)-1):
    start = dtPeriods2021[y]
    end = dtPeriods2021[y+1]
    time_interval = start, end
    dtIntervals2021.append(time_interval)

for y in range(len(dtPeriods2022)-1):
    start = dtPeriods2022[y]
    end = dtPeriods2022[y+1]
    time_interval = start, end
    dtIntervals2022.append(time_interval)

# Loop over Catch Crop Fields from GeoJSON
for i, field in enumerate(ccFields):

    # Sets time interval depending on
    start_date = str(int(field['properties']['Year'])) + "-08-01"
    end_date = str(int(field['properties']['Year'])) + "-11-01"
    time_interval = start_date, end_date

    # Converts geometry from GeoJSON to SentinelHub format
    geometry = Geometry(field['geometry'], crs=CRS.WGS84)
    
    # Create a search iterator to find avalable imagery for the field polygon
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        geometry = geometry,
        time = time_interval,
        fields = {"include": ["id", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
        )
    
    results = list(search_iterator)

    if field['properties']['Year'] == 2021:
        dtIntervals = dtIntervals2021
    elif field['properties']['Year'] == 2022:
        dtIntervals = dtIntervals2022
    else:
        print("Error")

    # Sets margin to filter acqusition time to avoid pictures from within the same time_difference
    time_difference = dt.timedelta(hours=1)

    # Use in case of last image in the last period
#    dtIntervals = [dtIntervals[8]]

    leastCCDatetimes = []
    leastCC = []

    for interval in dtIntervals:
        intervalResults = [x for x in results if (interval[0] < dt.datetime.fromisoformat(x['properties']['datetime']) < interval[1])]
        resultSort = sorted(intervalResults, key=lambda i: (i['properties']['eo:cloud_cover']))
        leastCCDatetimes.append(dt.datetime.fromisoformat(resultSort[0]['properties']['datetime']))
        leastCC.append(resultSort[0]['properties']['eo:cloud_cover'])

    field['properties']['imagePeriod'] = dtIntervals
    field['properties']['imageDateTime'] = leastCCDatetimes
    field['properties']['imageCloudCover'] = leastCC

    process_requests = []

    for timeStamp in leastCCDatetimes:
        
        request = SentinelHubRequest(
            evalscript = evalscriptAllBands_L2A,
            #evalscript = evalscriptRGB,
            input_data = [
                SentinelHubRequest.input_data(
                    data_collection = DataCollection.SENTINEL2_L2A,
                    time_interval = (timeStamp - time_difference, timeStamp + time_difference),
                    mosaicking_order = "leastCC"
                    )
                ],
            responses = [SentinelHubRequest.output_response("default", MimeType.TIFF)],
            geometry = geometry,
            config = config,            
            )
        process_requests.append(request)

    client = SentinelHubDownloadClient(config=config)
    
    download_requests = [request.download_list[0] for request in process_requests]
    
    data = client.download(download_requests)
        
    # Inserts images in list 
    fieldData.append(data)
    
    # Makes the program wait for X second to avoid download rate limit
    time.sleep(2)

    # Print run number to 
    print(f'Index processed: {idxStart + i}')

    # Saves images (numpy array) to disk
    if (i + 1) % k == 0:
        image = np.asarray(fieldData)

        print(np.shape(image))

        path_to_OutputFile = f'D:/Data/CatchCrop/All/ccImageIndex{idxStart + i + 1 - k}_{idxStart + i}.npy'
        
        with open(path_to_OutputFile, "wb") as fp:
            np.save(fp, image, allow_pickle=True, fix_imports=True)

        fPath = f'D:/Data/CatchCrop/All/ccFieldsProcessed{idxStart + i + 1 - k}_{idxStart + i}.npy'

        with open(fPath, "w") as fp:
            json.dump(ccFields, fp, default=str)

        fieldData = []

In [ ]:
# =============================================================================
# Print images
# =============================================================================

fieldImages = fieldData[0:2]

for i in range(len(fieldImages)):
    data = fieldImages[i]
    dTimes = dtIntervals
    gt = ccData['features'][i]['properties']['ccLabel']
    
    ncols, nrows = 3, ceil(9/3)

    fig, axis = plt.subplots(
        ncols=ncols, nrows = nrows, figsize = (10, 20), subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}
        )
    
    for idx, (image, timeStamp) in enumerate(zip(data, dTimes)):
        ax = axis[idx // ncols][idx % ncols]
        image = np.asarray(image)
        # Change from BGR to RGB
        image = np.flip(image[:,:,0:3], axis=-1)
        ax.imshow(np.clip(image * 2.5 / 10000, 0, 1))
#        ax.set_title(timeStamp.date().isoformat(), fontsize = 10)        
        ax.set_title(f"GroundTruth: {gt}. \n Time: {timeStamp}." , fontsize = 8)
        
    plt.tight_layout()

### Hente Datoer

In [ ]:
# =============================================================================
# Sentinel 2 - L2A
# =============================================================================
# Subset number of fields to process
k = 500

# len(ccFields) noAlt: 4163
# # len(ccFields) All: 7935
idxStart = 0
idxEnd = 7943

ccFields = ccData['features'][idxStart:idxEnd] # To test with

# Create a dict of processed fields
processedFields = {'ID': []}

fieldData = []

# Create a list of periods in 2021 and 2022
dtPeriods2021 = pd.date_range(start="2021-08-01T00:00:00Z",end="2021-11-01T00:00:00Z", periods = 10).to_pydatetime().tolist()
dtPeriods2022 = pd.date_range(start="2022-08-01T00:00:00Z",end="2022-11-01T00:00:00Z", periods = 10).to_pydatetime().tolist()

# Create a list of intervals in 2021 and 2022
dtIntervals2021 = []
dtIntervals2022 = []

for y in range(len(dtPeriods2021)-1):
    start = dtPeriods2021[y]
    end = dtPeriods2021[y+1]
    time_interval = start, end
    dtIntervals2021.append(time_interval)

for y in range(len(dtPeriods2022)-1):
    start = dtPeriods2022[y]
    end = dtPeriods2022[y+1]
    time_interval = start, end
    dtIntervals2022.append(time_interval)

# Loop over Catch Crop Fields from GeoJSON
for i, field in enumerate(ccFields):

    # Sets time interval depending on
    start_date = str(int(field['properties']['Year'])) + "-08-01"
    end_date = str(int(field['properties']['Year'])) + "-11-01"
    time_interval = start_date, end_date

    # Converts geometry from GeoJSON to SentinelHub format
    geometry = Geometry(field['geometry'], crs=CRS.WGS84)
    
    # Create a search iterator to find avalable imagery for the field polygon
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        geometry = geometry,
        time = time_interval,
        fields = {"include": ["id", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
        )
    
    results = list(search_iterator)

    if field['properties']['Year'] == 2021:
        dtIntervals = dtIntervals2021
    elif field['properties']['Year'] == 2022:
        dtIntervals = dtIntervals2022
    else:
        print("Error")

    # Sets margin to filter acqusition time to avoid pictures from within the same time_difference
    time_difference = dt.timedelta(hours=1)

    # Use in case of last image in the last period
#    dtIntervals = [dtIntervals[8]]

    leastCCDatetimes = []
    leastCC = []

    for interval in dtIntervals:
        intervalResults = [x for x in results if (interval[0] < dt.datetime.fromisoformat(x['properties']['datetime']) < interval[1])]
        resultSort = sorted(intervalResults, key=lambda i: (i['properties']['eo:cloud_cover']))
        leastCCDatetimes.append(dt.datetime.fromisoformat(resultSort[0]['properties']['datetime']))
        leastCC.append(resultSort[0]['properties']['eo:cloud_cover'])

    field['properties']['imagePeriod'] = dtIntervals
    field['properties']['imageDateTime'] = leastCCDatetimes
    field['properties']['imageCloudCover'] = leastCC


In [ ]:
# Path to GeoJSON
# path_to_OutputFile = "Data/ccInfoGDF.geojson"
path_to_OutputFile = "C:/Users/morte/OneDrive - Syddansk Universitet/Speciale2023/Data/ccInfoGDFtest.geojson" # GeoJSON med alle typer efterafgrøder
# path_to_OutputFile = "C:/Users/morte/OneDrive - Syddansk Universitet/Speciale2023/Data/ccNoAltInfoGDFtest.geojson" # GeoJSON kun med Målrettede Efterafgrøder

with open(path_to_OutputFile, "w") as fp:
    json.dump(ccFields, fp, default=str)

In [ ]:
# path_to_OutputFile = "C:/Users/morte/OneDrive - Syddansk Universitet/Speciale2023/Data/ccFieldsImages.geojson"

# with open(path_to_OutputFile, "w") as fp:
#     json.dump(ccData, fp)

ccGDF = gpd.GeoDataFrame.from_features(ccData["features"])

ccGDF.to_file('C:/Users/morte/OneDrive - Syddansk Universitet/Speciale2023/Data/ccFieldsImages.gpkg', driver='GPKG', mode="a")  



#ccGDF = gpd.GeoDataFrame.from_features(ccData["features"])

#ccGDF.to_parquet('C:/Users/morte/OneDrive - Syddansk Universitet/Speciale2023/Data/ccFieldsImages.parquet')


In [ ]:
geoFile = "C:/Users/morte/OneDrive - Syddansk Universitet/Speciale2023/Data/ccFieldsImagesCM.gpkg"

test = geoFile[0]['properties']['sentinel2_L2A']

In [ ]:
#df = gpd.read_parquet('C:/Users/morte/OneDrive - Syddansk Universitet/Speciale2023/Data/ccFieldsImages.parquet')



#data = list(ccData['features'][0]['properties']['sentinel2_L2A'].values())

image1 = np.asarray(data[1])

image1 = np.flip(image1[:,:,0:3], axis=-1)

print(image1)

plot_image(image1, factor=3.5 / 10000, vmax=1)


In [ ]:
# =============================================================================
# Print images
# =============================================================================

# To chance from BGR to RGB
#np.flip(data[:,:,1:3], axis=-1) 
#img = img[:,:,::-1]

for i in range(len(df)):
    data = list(df[i]['properties']['sentinel2_L2A'].values())
    dTimes = list(df[i]['properties']['sentinel2_L2A'].keys())
    gt = df[i]['properties']['ccLabel']
    
    ncols, nrows = 2, ceil(len(dTimes)/2)

    fig, axis = plt.subplots(
        ncols=ncols, nrows = nrows, figsize = (10, 20), subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}
        )
    
    for idx, (image, timeStamp) in enumerate(zip(data, dTimes)):
        ax = axis[idx // ncols][idx % ncols]
        image = np.asarray(image)
     #   image = image[:,:,1:4]
        ax.imshow(np.clip(image * 2.5 / 255, 0, 1))
#        ax.set_title(timeStamp.date().isoformat(), fontsize = 10)        
        ax.set_title(f"GroundTruth: {gt}. Time: {timeStamp}." , fontsize = 10)
        
    plt.tight_layout()

In [ ]:
# =============================================================================
# Save to disk
# =============================================================================

# Fejl kan vist løses med liste men det skal vi så også huske npår indlæses...
# https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable

#b = data[0].tolist()

#a_restored = np.asarray(b)

print(data[0])

path_to_OutputFile = "Data/ccFieldsImages.geojson"

with open(path_to_OutputFile, "w") as fp:
    json.dump(ccData, fp)

ccGDF = gpd.GeoDataFrame.from_features(ccData["features"])

ccGDF.to_file('Data/ccFieldsImages.shp')

ccGDF.to_file('Data/ccFieldsImages.gpkg', driver='GPKG')#, mode="a")  

geoFile = "C:/specialeData/ccFieldsImagesCM.gpkg"

ccFields = gpd.read_file(geoFile, driver='GPKG')

#gdf.to_file('dataframe.shp', mode="a")  

ccGDF.to_feather('Data/ccFieldsImages.feather')

df = gpd.read_feather("Data/ccFieldsImages.parquet")  

ccGDF.to_parquet('Data/ccFieldsImages.parquet')

df = gpd.read_parquet("Data/ccFieldsImages.parquet")

# =============================================================================
# Tensor
# =============================================================================

test = {str(dTimes[i]): data[i] for i in range(len(dTimes))}

for item in data:    
    tensor1 = tf.convert_to_tensor(item)
    print(tensor1)

tensor1 = tf.convert_to_tensor(data)

test = np.stack(data)

for i in range(len(data)):
    
    path = str("Data/images/test"+str(i))
    
    np.save(path, data[i], allow_pickle=True, fix_imports=True)


In [ ]:
# =============================================================================
# Sentinel 2 - L2A
# =============================================================================

# Subset number of fields to process
ccData['features'] = ccData['features'][0:4000] # To test with

#third_list = [field for field in first_list if i['name'] not in second_list]

runN = 0

# Create a dict of processed fields
processedFields = {'ID': []}

fieldData = ['']

# Loop over Catch Crop Fields from GeoJSON
for field in ccData['features']:

    print(field['properties']['Year'])


    # Converts geometry from GeoJSON to SentinelHub format
    geometry = Geometry(field['geometry'], crs=CRS.WGS84)
    
    # Create a search iterator to find avalable imagery for the field polygon
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        geometry = geometry,
        time = time_interval,
        fields = {"include": ["id", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
        )
    
    resultsL2A = list(search_iterator)

    # Create a search iterator to find avalable imagery for the field polygon
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L1C,
        geometry = geometry,
        time = time_interval,
        fields = {"include": ["id", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
        )
    
    resultsL1C = list(search_iterator)


  